<img src="http://imgur.com/1ZcRyrc.png" style="float: left; margin: 20px; height: 55px">

# Webscraping OpenTable with Selenium: Guided Lab

-
---

> *Note: this lab is intended to be instructor guided.*


In today's codealong lab, we will build a scraper using [urlib](https://docs.python.org/3/library/urllib.request.html#module-urllib.request) and [BeautifulSoup](https://www.crummy.com/software/BeautifulSoup/bs4/doc/). We will remedy some of the pitfalls of automated scraping by using a "headless" browser called Selenium.

You will be scraping OpenTable's DC listings. We're interested in knowing the restaurant's **name, location, price, cuisine, rating, and reviews.**

OpenTable provides all of this information on this given page: [Open table listings](https://www.opentable.com/chicago-restaurant-listings)

## 1. Inspect the elements of this page to assure we can find each of the bits of information in which we're interested.

## 2. Use `requests` and `BeautifulSoup` to read the contents of the HTML.

In [1]:
import pandas as pd
import numpy as np
import requests
from bs4 import BeautifulSoup
from selenium import webdriver
from time import sleep

url = 'https://www.opentable.com/washington-dc-restaurant-listings'

In [6]:
# set the url we want to visit
res = requests.get(url)

## 3. Use Beautiful Soup to convert the raw HTML into a soup object.

In [7]:
soup = BeautifulSoup(res.content, 'lxml' )

## 4. Extract the name of each restaurant.

Let's first find each restaurant name listed on the page we've loaded. How do we find the page location of the restaurant? 

> *Hint: we need to know where in the **html** the restaurant element is housed.*

### 4.A See if you can find the restaurant name on the page. Keep in mind there are many restaurants loaded on the page.

In [9]:
print(soup.prettify())

<!DOCTYPE html>
<html lang="en">
 <head>
  <meta charset="utf-8"/>
  <meta content="IE=9; IE=8; IE=7; IE=EDGE" http-equiv="X-UA-Compatible"/>
  <title>
   Restaurant Reservation Availability
  </title>
  <meta content="noindex,nofollow" name="robots"/>
  <link href="//components.otstatic.com/components/favicon/1.0.6/favicon/favicon.ico" rel="shortcut icon" type="image/x-icon"/>
  <link href="//components.otstatic.com/components/favicon/1.0.6/favicon/favicon-16.png" rel="icon" sizes="16x16"/>
  <link href="//components.otstatic.com/components/favicon/1.0.6/favicon/favicon-32.png" rel="icon" sizes="32x32"/>
  <link href="//components.otstatic.com/components/favicon/1.0.6/favicon/favicon-48.png" rel="icon" sizes="48x48"/>
  <link href="//components.otstatic.com/components/favicon/1.0.6/favicon/favicon-64.png" rel="icon" sizes="64x64"/>
  <link href="//components.otstatic.com/components/favicon/1.0.6/favicon/favicon-128.png" rel="icon" sizes="128x128"/>
  <link href="//components.otstatic.

### 4.B See any issues here?

**ANSWER**:

## 5. Enter Selenium - resolve the javascript issue using the driver and find the bookings.

What we can do in this case is:
1. Request that the page load
2. Wait some amount of time
3. Grab the source html from the page 

Because the page should believe I'm visiting from a live connection on a browser client, the JavaScript should render to be a part of the page source. I can then grab the page source.

**Once you have the HTML with the javascript rendered, repeat the processes above.**

In [10]:
from selenium import webdriver

# uncomment below for macos
driver = webdriver.Chrome(executable_path="./chromedriver/macos/chromedriver") 

# uncomment below for windows 
#  driver = webdriver.Chrome(executable_path="./chromedriver/windows/chromedriver.exe")

In [11]:
driver.get(url)

In [12]:
driver.page_source

'<!DOCTYPE html><html xmlns="http://www.w3.org/1999/xhtml" lang="en"><head><meta charset="utf-8" /><meta http-equiv="X-UA-Compatible" content="IE=9; IE=8; IE=7; IE=EDGE" /> <title>Washington, D.C. Area Restaurants List | OpenTable</title>  <meta name="description" content="Find Washington, D.C. Area restaurants. Search by location, cuisine, or price to refine restaurant results in the Washington, D.C. Area area." />   <meta name="robots" content="noindex,nofollow" />      <link rel="shortcut icon" href="//components.otstatic.com/components/favicon/1.0.6/favicon/favicon.ico" type="image/x-icon" /><link rel="icon" href="//components.otstatic.com/components/favicon/1.0.6/favicon/favicon-16.png" sizes="16x16" /><link rel="icon" href="//components.otstatic.com/components/favicon/1.0.6/favicon/favicon-32.png" sizes="32x32" /><link rel="icon" href="//components.otstatic.com/components/favicon/1.0.6/favicon/favicon-48.png" sizes="48x48" /><link rel="icon" href="//components.otstatic.com/compon

## 6. Repeat the process above, and let's grab location as well 

In [16]:
soup = BeautifulSoup(driver.page_source, 'lxml')

names = []
for name in soup.find_all('span',{'class':'rest-row-name-text'})[:100]:
    names.append(name.text)

locs = []
for location in soup.find_all('span', {'class': 'rest-row-meta--location rest-row-meta-text sfx1388addContent'})[:100]:
    locs.append(location.text)

for name, location in zip(names, locs):
    print(f'{name} in {location}.')

BlackSalt in Palisades Northwest.
Ambar - Arlington in Arlington.
Et Voila in Palisades Northwest.
Ruffino's - Arlington in Arlington.
Bistro Aracosia in Palisades Northwest.
SER in Arlington.
The Melting Pot - Arlington VA in Arlington.
True Food Kitchen - Arlington, Ballston Quarter in Arlington.
Maya Bistro in Arlington.
Medium Rare - Arlington in Arlington.
Lyon Hall in Arlington.
Uncle Julio's - Arlington/ Ballston in Arlington.
Gyu-Kaku - Arlington, VA in Arlington.
Lupo Verde Osteria Palisades in Palisades Northwest.
CIRCA at Clarendon in Arlington.
Essy's Carriage House in Arlington.
Cava Mezze - Clarendon in Arlington.
The Liberty Tavern in Arlington.
Green Pig Bistro in Arlington.
Filomena Ristorante in Georgetown.
Mussel Bar & Grille - Arlington in Arlington.
Grand Cru Wine Bar & Bistro in Arlington.
Rustico - Ballston in Arlington.
Pamplona in Arlington.
Tupelo Honey - Arlington in Arlington.
Buena Vida - Arlington in Arlington.
Baba in Arlington.
Fiola Mare in Georgetown.


## 7. Get the price (dollar signs) for each restaurant.

The price is number of dollar signs on a scale of one to four for each restaurant. We'll follow the same process.

In [18]:
# A:
for d in soup.find_all('i', {'class': 'pricing--the-price'})[:100]:
    print(len(d.text.replace(' ','')))

3
2
3
2
2
2
3
2
2
2
2
2
2
2
2
3
2
2
2
3
2
2
2
2
2
2
2
4
3
3
2
2
2
2
2
2
2
2
2
4
3
2
2
2
2
2
2
2
3
2
2
2
2
2
2
2
3
2
4
2
2
4
2
3
3
2
4
2
2
2
3
2
2
2
2
2
2
2
3
4
2
2
3
2
2
2
2
3
3
4
4
3
2
4
4
2
2
3
4
2


## 8. Get the name, location, price, cuisine, rating, and reviews for reach restaurant.

Let's go through this together a bit.

In [20]:
# get the name
l_name = []
for i in soup.find_all('span', {'class': 'rest-row-name-text'}):
    l_name.append(i.text)

# get the location
l_loc = []
for i in soup.find_all('span',{'class':'rest-row-meta--location rest-row-meta-text sfx1388addContent'}):
    l_loc.append(i.text)

# get the pricing
l_pr = []
for d in soup.find_all('i', {'class': 'pricing--the-price'}):
    l_pr.append(len(d.text.replace(' ','')))

# get the cuisine type
l_cuisine = []
for c in soup.find_all('span', {'class': 'rest-row-meta--cuisine rest-row-meta-text sfx1388addContent'}):
    l_cuisine.append(c.text)
    
# get the rating out of 5
l_rating = []
for i in soup.find_all('div', {'class': 'star-rating-score'}):
    l_rating.append(float(i.attrs['aria-label'].split(' ')[0]))
    
# get the number of reviews
l_reviews = []
for i in soup.find_all('a',{'class':'review-link'}):
    l_reviews.append(int(i.find('span').text.strip('()')))

# store it all in a dictionary of lists and make it into a dataframe
info_dict = {'name': l_name,
             'location': l_loc,
             'price': l_pr,
             'cuisine': l_cuisine,
             'rating': l_rating,
             'reviews': l_reviews,
            }

pd.DataFrame(info_dict).head()

,name,location,price,cuisine,rating,reviews
0,BlackSalt,Palisades Northwest,3,Seafood,4.7,6009
1,Ambar - Arlington,Arlington,2,Tapas / Small Plates,4.6,2057
2,Et Voila,Palisades Northwest,3,French,4.7,1464
3,Ruffino's - Arlington,Arlington,2,Italian,4.1,131
4,Bistro Aracosia,Palisades Northwest,2,Afghan,4.8,1387


In [21]:
counter = 1 

while counter < 10:
    print(counter)
    time.sleep(10)

1


NameError: name 'time' is not defined

### 9. Can we get all of the items we want from the page in a single `find_all`?

To be most efficient, we want to only do a single loop for each entry on the page. That means we want to find what element all of other other elements (name, location, price, bookings) is housed within. Where on the page is each entry located?

In [ ]:
# A:


In [ ]:
# initialize dataframe
df = pd.DataFrame(columns=['name', 'location', 'price', 'cuisine','rating','reviews'])

# one big for loop!
    

df.head()

## 10. Does every single entry have each element we want?

**ANSWER**:

## BONUS

## 11. Putting it all together in a function.

**Loop through each entry. For each entry:**
1. Grab the relevant information we want (name, location, price, bookings). 
2. Produce a dataframe with the columns "name","location","price","bookings" that contains the 100 entries we would like.

In [ ]:
def restaurant_grabber(soup, df):
    pass

## 12. Use selenium to loop through at least 5 pages and grab that information as well. 

In [ ]:
df_chi = pd.DataFrame(columns=['name', 'location', 'price', 'cuisine','rating','reviews'])
url = 'https://www.opentable.com/chicago-restaurant-listings'
driver = webdriver.Chrome(executable_path="./chromedriver/macos/chromedriver")
driver.get(url)


for _ in range(6):
    sleep(5)
    soup_object = BeautifulSoup(driver.page_source, 'lxml')
    restaurant_grabber(soup_object, df_chi)
    next_button = driver.find_element_by_link_text('Next')
    next_button.click()

driver.close()

df_chi.head()

In [ ]:
df_chi[df_chi.duplicated()]

In [ ]:
df_chi.shape

## Examples of Other Neat-o Selenium Stuff

In [ ]:
# you can also use selenium to make clicks!

url = 'https://www.opentable.com/chicago-restaurant-listings'
driver = webdriver.Chrome(executable_path="./chromedriver/macos/chromedriver")
driver.get(url)
driver.implicitly_wait(2)
price4 = driver.find_element_by_xpath('//*[@id="PriceBands-filter-items"]/ul/li[3]/label')

price4.click()
sleep(5)
driver.close()

In [ ]:
# can also type things into a search bar
from selenium.webdriver.common.keys import Keys

driver = webdriver.Chrome(executable_path="./chromedriver/macos/chromedriver") 
driver.get("http://www.python.org")
sleep(5)
assert "Python" in driver.title
elem = driver.find_element_by_name("q")
elem.clear()
elem.send_keys("pycon")
sleep(5)
elem.send_keys(Keys.RETURN)
sleep(5)
assert "No results found." not in driver.page_source
sleep(5)
driver.close()

## Additional resources

---

The above example (and many others) are available in the Selenium docs: http://selenium-python.readthedocs.io/getting-started.html

What is especially important is exploring functionality like locating elements: http://selenium-python.readthedocs.io/locating-elements.html#locating-elements

FAQ:
http://selenium-python.readthedocs.io/faq.html